In [1]:
!nvidia-smi

Thu Dec 26 18:21:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip -q install sentence_transformers langchain tiktoken chromadb pypdf transformers langchain_community
!pip -q install accelerate bitsandbytes sentencepiece Xformers

In [3]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader, DirectoryLoader

import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [4]:
!wget https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip -O stock_market_june_2023.zip
!unzip stock_market_june_2023.zip

--2024-12-26 18:22:16--  https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip [following]
--2024-12-26 18:22:16--  https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10638 (10K) [application/zip]
Saving to: ‘stock_market_june_2023.zip’

stock_market_june_2 100%[===================>]  10.39K  --.-KB/s    in 0.001s  

2024-12-26 18:22:17 (7.44 MB/s) - ‘stock_market_june_2023.zip’ saved [10638/10638]


In [5]:
loader = DirectoryLoader('/content/stock_market_june_2023', glob='./*.txt', loader_cls=TextLoader)

documents = loader.load()

In [6]:
#documents
len(documents)

9

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

In [8]:
len(texts)
#texts[0]

49

In [9]:
model = 'tiiuae/falcon-7b-instruct'

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,  # Set max tokens for the output
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map='cuda'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
llm = HuggingFacePipeline(pipeline=pipeline)

model_name = 'intfloat/e5-large-v2'
hf = HuggingFaceEmbeddings(model_name=model_name)

<ipython-input-11-dfd8260087bf>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipeline)
<ipython-input-11-dfd8260087bf>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceEmbeddings(model_name=model_name)


# Create DB

In [12]:
persist_directory = 'db'  #directory where embeddings will be stored

# convert documents to embeddings using hugging face embedding and then specifies the path where the vector database of those embeddings are to be stored
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=hf,
                                 persist_directory=persist_directory)

In [13]:
vectordb.persist()  #persists the embedding database to disk

vectordb = Chroma(persist_directory=persist_directory,  # load the stored vector embeddings to chroma db
                  embedding_function=hf)  # use embedding function to query and search

<ipython-input-13-fb36a474b514>:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()  #persists the embedding database to disk
<ipython-input-13-fb36a474b514>:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory,  # load the stored vector embeddings to chroma db


In [14]:
# init retriever object with 3 search results
retriever = vectordb.as_retriever(search_kwargs={'k':3})

In [15]:
# initialize Q & A langchain's chain
# user query is used to retrieve the most relevant 3 documents from the db
# all of the 3 relevant documents are concatenated (stuff)
# then it is passed to the llm along with the user query
# llm return the answer
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=retriever,
                                       return_source_documents=True)

In [16]:
def process_llm_response(llm_response):
  print(llm_response['result'])

  for source in llm_response['source_documents']:
    print(source.metadata['source'])

In [19]:
query = "What were the reasons behind Shopify's decision to lay off a portion of its workforce?"
response = qa_chain(query)
process_llm_response(response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

To adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of approximately 20% and selling its logistics business to Flexport. Although these changes incur near-term expenses, such as restructuring and impairment charges, they will ultimately reduce overhead costs and make Shopify more asset-light in the long run.

To adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of approximately 20% and selling its logistics business to Flexport. Although these changes incur near-term expenses, such as restructuring and impairment charges, they will ultimately reduce overhead costs and make Shopify more asset-light in the long run.

To adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of 

In [20]:
query = "What were the reason behind Nvidia's stock growth?"
response = qa_chain(query)
process_llm_response(response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Nvidia’s shares have seen a massive surge since the beginning of 2023, indicating that the market recognizes the company’s substantial potential in the AI industry. While some may consider the stock to be pricy, Nvidia has a lot of things going for it to help justify the premium price tag.

Nvidia’s shares have seen a massive surge since the beginning of 2023, indicating that the market recognizes the company’s substantial potential in the AI industry. While some may consider the stock to be pricy, Nvidia has a lot of things going for it to help justify the premium price tag.

Nvidia’s shares have seen a massive surge since the beginning of 2023, indicating that the market recognizes the company’s substantial potential in the AI industry. While some may consider the stock to be pricy, Nvidia has a lot of things going for it 

In [ ]:
# mistral 7B